In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
from sklearn.preprocessing import StandardScaler
from statsmodels.graphics.gofplots import qqplot
from scipy.stats import boxcox, weibull_min, exponweib, shapiro
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.stattools import adfuller
import numpy as np
from pmdarima.arima import auto_arima, ARIMA
from scipy.special import inv_boxcox
plt.style.use('ggplot')

In [ ]:
def make_pred_plot(df, target_col_name='real', pred_start=0, pred_end=50, pred_interval=2, 
                   xlim_low=0, xlim_up=200, get_errors=True, scale=True):
    '''
    get a df has ['real', 's1', 's2', ...] in columns to make plot
    '''

    
    pred_steps = len(df.drop(columns=target_col_name).columns)
    print("pred_steps: {}".format(pred_steps))
    def evaluate_pred(df, target_col_name):

        
        real = np.array([df.real[i:pred_steps+i].values for i in range(len(df)-pred_steps + 1 )])

        preds = df.drop(columns=target_col_name).values[:len(df)-pred_steps+1,:]
        errors = real - preds

        rmse = (np.mean(errors**2)*pred_steps)**(1/2)
        mae = np.mean(abs(errors))*pred_steps
        print('RMSE: {}'.format(rmse))
        print('MAE: {}'.format(mae))
        return errors
    
    errors = evaluate_pred(df, target_col_name=target_col_name)    
    
    if scale:
        ss = StandardScaler()
        df = pd.DataFrame(ss.fit_transform(df), columns=df.columns)
    
    fig = figure(num=None, figsize=(20, 6))
    ax = fig.add_subplot(111)
    ax.plot(df[target_col_name], 'g')

    for pred_count in range(pred_start, pred_end, pred_interval):
        ax.plot([None for i in range(pred_count)] + list(df.drop(columns=target_col_name).iloc[pred_count,:].values), 
                'r--.', linewidth=0.5)

    ax.legend(['real','predicted'], prop={'size':15})
    ax.set_xlim(xlim_low, xlim_up)
    ax.set_xlabel('Step')
    if scale:
        ax.set_ylabel('Scaled_wind_speed')
    else:
        ax.set_ylabel('Wind_speed')
    plt.show()
    if get_errors:
        return fig, ax, errors
    else:
        return fig, ax

def grid_search(df_fit, order_list):
    for i, o in enumerate(order_list):
        try:
            model = ARIMA(order=o, method='css-mle')
            model_fit = model.fit(y = df_fit.values.flatten())
            if i == 0:
                best_aicc = model_fit.aicc()
                best_order = o

            elif best_aicc > model_fit.aicc():
                best_aicc = model_fit.aicc()
                best_order = o
        except:
            pass
        print('order:{}, aicc:{}, bic:{}'.format(o, model_fit.aicc(), model_fit.bic()))
    return best_order

def make_init_visual(series, show_acf=False):
#     fig = plt.figure(constrained_layout=True, figsize=(10, 8))
#     gs = fig.add_gridspec(2, 2)
#     ax1 = fig.add_subplot(gs[0, :])
#     ax2 = fig.add_subplot(gs[1, 0])
#     ax3 = fig.add_subplot(gs[1, 1])
    
#     ax1.plot(series, 'c', linewidth=0.8)
#     ax1.set_xlabel('Second', fontsize=12)
#     ax1.set_ylabel('Wind_speed', fontsize=12)
#     ax2.hist(series, bins=30)
#     ax2.set_xlabel('Wind_speed', fontsize=12)
#     ax3 = qqplot(series, line='q', ax = ax3)
    
    fig = plt.figure(constrained_layout=True, figsize=(10, 3))
    gs = fig.add_gridspec(1, 2)
    ax1 = fig.add_subplot(gs[0, 0])
    ax2 = fig.add_subplot(gs[0, 1])
    
    ax1.hist(series, bins=30)
    ax1.set_xlabel('Scaled_wind_speed (m/s)', fontsize=12)
    ax2 = qqplot(series, line='q', ax = ax2)
    
    t, p = shapiro(series)
    print("Shapiro_test_statistic:{}, p_value:{}".format(t, p))
    return fig

In [ ]:
# load data and plot
df = pd.read_csv('data/v_hub_filt.csv')

fig = plt.figure(figsize=(20, 5))
plt.plot(df.v_hub_filt, 'c', linewidth=0.5)

In [ ]:
# train test split
split_point = int(len(df)*0.8)

df_train = df.v_hub_filt[:split_point]
df_test = df.v_hub_filt[split_point:]

In [ ]:
fig = make_init_visual(df_train)

In [ ]:
#boxcox transformation
l = boxcox(df_train)[1]
print('boxcox_lambda:{}'.format(l))
df_train = boxcox(df_train)[0]
fig = make_init_visual(df_train)
df_train = pd.DataFrame(df_train)
# fig.savefig('thesis/figures/ARIMA_boxcox_trans.eps', format='eps', dpi=1200)

In [ ]:
fig = plot_acf(df_train.diff().dropna(), lags=60, alpha=0.05)
# plt.savefig('thesis/figures/acf.eps', format='eps', dpi=1200)


fig = plot_pacf(df_train.diff().dropna(), lags=60, alpha=0.05)
# plt.savefig('thesis/figures/pacf.eps', format='eps', dpi=1200)

In [ ]:
fig = plt.figure(figsize=(20, 5))
plt.plot(df_train.diff().dropna(), 'c', linewidth=0.5)
plt.xlabel('Second', fontsize=12)
plt.ylabel('Differenced_wind_speed (m/s)', fontsize=12)
# plt.savefig('thesis/figures/differenced_wind_speed.eps', format='eps', dpi=1200)

In [ ]:
adfuller(df_train.diff().dropna().values.flatten())

# Find best ARIMA(p, d, q)

In [ ]:
best_ARIMA = auto_arima(df_train, start_p=13, start_q=0, max_p=16, max_q=1, m=1,
                        seasonal=False, trace=True, d=2, max_order=None,
                        suppress_warnings=True, error_action="ignore",
                        stepwise=False)

In [ ]:
def grid_search(df_fit, order_list, exogenous):
    for i, o in enumerate(order_list):
        try:
            if i == 0:
                best_aicc = model_fit.aicc()
                best_order = o

            elif best_aicc > model_fit.aicc():
                best_aicc = model_fit.aicc()
                best_order = o
        except:
            pass
        
        print('order:{}, aicc:{}, bic:{}'.format(o, model_fit.aicc(), model_fit.bic()))
    return best_order

In [ ]:
from statsmodels.tsa.arima_model import ARIMA
model = ARIMA(endog = df_train.values.flatten(), exog=exogenous, order=[1, 1, 1])
model_fit = model.fit(disp=0)
# model_fit = model.fit(endog = df_train.values.flatten(), exog=exogenous)
# print('aicc:{}, bic:{}'.format(model_fit.aicc(), model_fit.bic()))


In [ ]:
model_fit.aic

In [ ]:
exogenous=df.vdir_hub[:split_point].values.reshape(-1,1)
p_list = np.arange(10,11).tolist()
q_list = np.arange(0,2).tolist()
d = 1
# # orders = [(p, d, q) for p in p_list for q in q_list]
# q = 1
orders = [(p, d, q) for p in p_list for q in q_list]
orders
grid_search(df_train, [1, 1, 0], exogenous=exogenous)

# ARIMAX

In [ ]:
exogenous=df.vdir_hub[:split_point].values.reshape(-1,1)
model = ARIMA(order=(12,1,1), method='mle', maxiter=100)
model_fit = model.fit(y = df_train.values.flatten(), exogenous=exogenous)

In [ ]:
diagnose_plot = model_fit.plot_diagnostics(figsize=(8, 8))

In [ ]:
# predict
exogenous_test = df.vdir_hub[split_point:].values.reshape(-1,1)
preds = []
for i, v in enumerate(df_test.values):
    if i == 0:
        pred = model_fit.predict(5, exogenous=np.repeat(df.vdir_hub[:split_point].values[-1], 5).reshape(-1,1))
    else:
        pred = model_fit.predict(5, exogenous=np.repeat(exogenous_test[i-1],5).reshape(-1,1))
    pred = inv_boxcox(pred, l)
    preds.append(pred)
    new_observation = boxcox(v, l)
    model_fit.add_new_observations(y=np.array(new_observation).reshape(1,), 
                                   exogenous=exogenous_test[i].reshape(-1,1))

In [ ]:
dict_pred = {'s{}'.format(i+1):list(np.array(preds)[:,i]) for i in range(np.array(preds).shape[1])}
dict_pred['real'] = list(df_test.values)
df_pred = pd.DataFrame.from_dict(dict_pred)
fig, ax, arimax_error = make_pred_plot(df_pred.dropna(), target_col_name='real', pred_start=900, 
                                       pred_end=1000, pred_interval=2, xlim_low=900, xlim_up=1000,
                                       scale=True)
ax.set_title('ARIMA')
ax.set_ylim(0,4)
fig

In [ ]:
np.save('arimax_error', arimax_error)

In [ ]:
ax.set_title('')
ax.set_ylim(-1,3)
fig
fig.savefig('thesis/figures/ARIMA_result.eps', format='eps', dpi=1200)

In [ ]:
df_pred

In [ ]:
StandardScaler